In [1]:
#| default_exp fastlite

In [2]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from fastlite import database
from monsterui.all import * 

from monsterui.all import *
from pathlib import Path
from fastcore.utils import *

In [3]:
#| export
import fitz  # PyMuPDF
from dataclasses import dataclass
from datetime import datetime
from collections import Counter

In [4]:
highlights = L()
with fitz.open(path) as doc:
    for page_num, page in enumerate(doc, 1):
        for annot in page.annots():
            if annot.type[0] == 8:  # 8 is highlight annotation
                # Use the annotation's rect directly
                rect = annot.rect
                text = page.get_text("text", clip=rect).strip()
                
                highlights.append(Highlight(
                    text=text,
                    page=page_num,
                    color=annot.colors.get('stroke', [1, 1, 0]),
                    created=datetime.now()
                ))

NameError: name 'path' is not defined

In [ ]:
highlights[0]

In [ ]:
#| export

# Create/open database
db = database('highlights.db')

# Define your table structure - fastlite works great with dataclasses
class Highlight:
    id: int
    text: str
    page: int
    color: str  # We'll store as string for simplicity
    pdf_file: str  # Track which PDF it came from
    created: str  # ISO format datetime string

# Create the table (transform=True allows schema updates)
highlights_db = db.create(Highlight, pk='id', transform=True)

In [ ]:
#| export
def save_highlights(pdf_path, highlights):
    "Save extracted highlights to database"
    pdf_name = Path(pdf_path).name
    for h in highlights:
        highlights_db.insert(
            text=h.text,
            page=h.page,
            color=str(h.color),  # Convert tuple to string
            pdf_file=pdf_name,
            created=h.created.isoformat()
        )

In [ ]:
save_highlights(path, highlights)

In [ ]:
highlights_db()[0]

## UI

In [ ]:
#| export
def highlight_card(h):
    # Parse color and convert to CSS
    color_rgb = eval(h.color)
    # Convert RGB (0-1 range) to CSS rgb format (0-255 range)
    r, g, b = [int(c * 255) for c in color_rgb]
    color_style = f"background-color: rgb({r}, {g}, {b});"
    
    return Card(
        # Color indicator strip - using inline style instead of Tailwind classes
        Div(cls="absolute left-0 top-0 bottom-0 w-1", style=color_style),
        # Main content with padding to avoid color strip
        Div(
            P(h.text, cls=TextPresets.md_weight_sm),
            cls="pl-4"
        ),
        footer=DivFullySpaced(
            DivHStacked(
                UkIcon('file-text', height=16),
                P(h.pdf_file, cls=TextPresets.muted_sm)
            ),
            DivHStacked(
                UkIcon('book-open', height=16),
                P(f"Page {h.page}", cls=TextPresets.muted_sm)
            )
        ),
        cls=CardT.hover,
        style="position: relative;"
    )

## Routes

In [ ]:
#| export
# Initialize FastHTML app with MonsterUI theme
app, rt = fast_app(hdrs=Theme.blue.headers())

In [ ]:
#| export
@app.route('/')
def index():
    pdf_counts = Counter(h.pdf_file for h in highlights_db())

    pdf_cards = [
        Card(
            H3(pdf_name),
            P(f'{count} highlights', cls=TextPresets.muted_sm),
            footer = Button("View Highlights",
                                     hx_get=f"/pdf/{pdf_name}",
                                     hx_target="#main-content",
                                     cls=ButtonT.primary),
            cls=CardT.hover
        )
        for pdf_name, count in pdf_counts.items()
    ]  

    # Upload form
    upload_form = Card(
        H3("Upload New PDF"),
        Form(
            Input(type="file", name="pdf_file", accept=".pdf"),
            Button("Upload & Analyze", type="submit", cls=ButtonT.primary),
            hx_post="/upload",
            hx_target="#main-content",
            enctype="multipart/form-data"
        )
    )

    return Titled(
        "PDF Highlights Manager",
        Div(
            upload_form,
            Divider(),
            H2("Your PDFs"),
            Grid(*pdf_cards, cols=3),
            id="main-content",
            cls="space-y-6"
        )
    )

In [ ]:
#| export
@app.route('/pdf/{pdf_name}')
def view_pdf(pdf_name:str):
    highlights = highlights_db(where=f"pdf_file='{pdf_name}'", order_by='page')
    cards = [highlight_card(h) for h in highlights]

    return Div(
        DivFullySpaced(
            H2(pdf_name),
            Button("← Back", hx_get="/", hx_target="#main-content", cls=ButtonT.ghost)
        ),
        Div(*cards, cls='space-y-4')
    )

In [ ]:
#| export
@app.route('/upload', methods=['POST'])
async def upload(pdf_file: UploadFile):
    content = await pdf_file.read()

    new_highlights = extract_highlights(content)
    save_highlights(pdf_file.filename, new_highlights)
    
    return Div(
        Alert(
            f"Successfully extracted {len(new_highlights)} highlights from {pdf_file.filename}!",
            cls=AlertT.success
        ),
        # Trigger a refresh to show the new PDF card
        Div(hx_get="/", hx_trigger="load delay:2s", hx_target="#main-content")
    )

In [ ]:
server.stop()

In [ ]:
server = JupyUvi(app)

In [ ]:
HTMX()

In [5]:
from nbdev.export import nb_export
nb_export('fastlite.ipynb')